# Notebook to perform every action in one go

In [46]:
import numpy as np
import cv2 as cv
import tensorflow as tf
import plotly.express as px
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import load_img, img_to_array # type: ignore

from src.xml_parser import parse_xml_files, save_labels_to_csv, extract_image_filenames
from src.data_verifcation import verify_bounding_box
from src.data_processing import process_data
from src.model_training import build_and_train_model

### Data Extraction and XML Parsing

In [2]:
label_dictionary = parse_xml_files(r'..\input\number-plates\annotations\*.xml')

In [3]:
df = save_labels_to_csv(label_dictionary, r'..\output\labels.csv')

In [4]:
image_path = extract_image_filenames(df)

In [5]:
df

,filepath,xmin,xmax,ymin,ymax
0,..\input\number-plates\annotations\Cars0.xml,226,419,125,173
1,..\input\number-plates\annotations\Cars1.xml,134,262,128,160
2,..\input\number-plates\annotations\Cars10.xml,140,303,5,148
3,..\input\number-plates\annotations\Cars100.xml,175,214,114,131
4,..\input\number-plates\annotations\Cars101.xml,167,240,202,220
...,...,...,...,...,...
428,..\input\number-plates\annotations\Cars95.xml,278,407,182,263
429,..\input\number-plates\annotations\Cars96.xml,133,261,126,160
430,..\input\number-plates\annotations\Cars97.xml,98,297,102,191
431,..\input\number-plates\annotations\Cars98.xml,85,247,196,259


### Data Verification and Visualization

In [6]:
file_path = image_path[120]
xmin, xmax, ymin, ymax = df.iloc[120, 1:5]
verify_bounding_box(file_path, xmin, ymin, xmax, ymax)

![Prototyping Test 1](../output/examples/prototype_test_1.png)

In [7]:
file_path = image_path[124]
xmin, xmax, ymin, ymax = df.iloc[124, 1:5]
verify_bounding_box(file_path, xmin, ymin, xmax, ymax)

![Prototyping Test 2](../output/examples/prototype_test_2.png)

In [8]:
file_path = image_path[5]
xmin, xmax, ymin, ymax = df.iloc[5, 1:5]
verify_bounding_box(file_path, xmin, ymin, xmax, ymax)

![Prototyping Test 3](../output/examples/prototype_test_3.png)

### Performing Data Processing

In [9]:
x_train, x_test, y_train, y_test = process_data(df)

In [15]:
x_train.shape

(346, 224, 224, 3)

In [16]:
x_test.shape

(87, 224, 224, 3)

In [17]:
y_train.shape

(346, 4)

In [18]:
y_test.shape

(87, 4)

In [20]:
print(x_train.shape)  # Should be (num_samples, height, width, channels)
print(y_train.shape)  # Should be (num_samples, num_classes) for classification tasks

print(x_test.shape)   # Should be (num_samples, height, width, channels)
print(y_test.shape)   # Should be (num_samples, num_classes)


(346, 224, 224, 3)
(346, 4)
(87, 224, 224, 3)
(87, 4)


### Model Training

In [21]:
# model = build_and_train_model(x_train, y_train, x_test, y_test)
# model.save(r'..\output\object_detection.h5')

In [22]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False, input_tensor=Input(shape=(224,224,3)))

headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)

model = Model(inputs=inception_resnet.input,outputs=headmodel)

In [23]:
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 111, 111,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 111, 111,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 109, 109,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 109, 109,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 109, 109,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 109, 109,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 54, 54,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 54, 54,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 54, 54,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 54, 54,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 52, 52,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 52, 52,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 52, 52,    │          0 │ batch_normalizat

 Total params: 73,663,490 (281.00 MB)

 Trainable params: 73,602,946 (280.77 MB)

 Non-trainable params: 60,544 (236.50 KB)

In [25]:
tfb = TensorBoard('object_detection')
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=10,
                    validation_data=(x_test,y_test),callbacks=[tfb])

Epoch 1/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 204s 6s/step - loss: 0.0029 - val_loss: 0.0144
Epoch 2/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 235s 7s/step - loss: 0.0024 - val_loss: 0.0148
Epoch 3/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 231s 7s/step - loss: 0.0012 - val_loss: 0.0156
Epoch 4/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 232s 7s/step - loss: 0.0012 - val_loss: 0.0158
Epoch 5/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 247s 7s/step - loss: 0.0014 - val_loss: 0.0163
Epoch 6/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 205s 6s/step - loss: 0.0011 - val_loss: 0.0154
Epoch 7/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 160s 5s/step - loss: 9.1579e-04 - val_loss: 0.0157
Epoch 8/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 159s 5s/step - loss: 0.0022 - val_loss: 0.0196
Epoch 9/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 165s 5s/step - loss: 0.0019 - val_loss: 0.0169
Epoch 10/10
35/35 ━━━━━━━━━━━━━━━━━━━━ 168s 5s/step - loss: 0.0016 - val_loss: 0.0145


In [29]:
model.save(r'..\output\object_detection.keras')

In [85]:
model = tf.keras.models.load_model(r'..\output\object_detection_e10.keras')

c:\Users\Asif Sayyed\Documents\GitHub\LicensePlateDetection\venv\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning:

Skipping variable loading for optimizer 'rmsprop', because it has 496 variables whereas the saved optimizer has 990 variables. 



In [95]:
path = r'../input/number-plates/images/Cars82.png'
image = load_img(path) # PIL object
image = np.array(image,dtype=np.uint8) # 8 bit array (0,255)
image1 = load_img(path,target_size=(224,224))
image_arr_224 = img_to_array(image1)/255.0  # Convert into array and get the normalized output

h,w,d = image.shape
print('Height of the image =',h)
print('Width of the image =',w)

Height of the image = 151
Width of the image = 400


In [96]:
fig = px.imshow(image)
fig.update_layout(width=700, height=500,  margin=dict(l=10, r=10, b=10, t=10), xaxis_title='Figure 13 - TEST Image')

In [97]:
image_arr_224.shape

(224, 224, 3)

In [98]:
test_arr = image_arr_224.reshape(1,224,224,3)
test_arr.shape

(1, 224, 224, 3)

### De-normalizing the output

In [99]:
# Make predictions
coords = model.predict(test_arr)
coords

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 301ms/step


array([[0.4441456 , 0.62544656, 0.36615944, 0.53595674]], dtype=float32)

In [100]:
# Denormalize the values
denorm = np.array([w,w,h,h])
coords = coords * denorm
coords

array([[177.65823603, 250.1786232 ,  55.2900753 ,  80.9294678 ]])

### Bounding box

In [101]:
coords = coords.astype(np.int32)
coords

array([[177, 250,  55,  80]])

In [102]:
xmin, xmax,ymin,ymax = coords[0]
pt1 =(xmin,ymin)
pt2 =(xmax,ymax)
print(pt1, pt2)

(177, 55) (250, 80)


In [103]:
cv.rectangle(image,pt1,pt2,(0,255,0),3)
fig = px.imshow(image)
fig.update_layout(width=700, height=500, margin=dict(l=10, r=10, b=10, t=10))